In [19]:
import warnings
warnings.filterwarnings('ignore')

## import libraries

In [20]:
from pickle import load
from numpy import argmax
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
 


## Configurations

In [25]:
# initialize the dataset variable with 1 and 2 for flickr8k and flickr30k respectively
dataset = 1
max_length = 34 if dataset == 1 else 70
# initialize the tokenizer for selected dataset
tokenizer_path = 'tokenizer/flickr8k_tokenizer.pkl'
image_path = 'image.jpg'

## VGG 16
### utilities

In [26]:
# extract features from each photo in the directory
def extract_features(filename):
	# load the model
	model = VGG19()
	# re-structure the model
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	# load the photo
	image = load_img(filename, target_size=(224, 224))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the VGG model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature
 
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text
 


### load tokenizer, model & image

In [ ]:
# load the tokenizer
tokenizer = load(open(tokenizer_path, 'rb'))
# load the model
model = load_model('models/flickr8k/vgg16/saved-model-05.h5')
# load and prepare the photograph
photo = extract_features(image_path)

### predict & show caption

In [5]:
# generate description
description = generate_desc(model, tokenizer, photo, max_length)
print(description)

startseq two children are playing in the grass endseq


## VGG 19
### utilities

In [16]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
# extract features from each photo in the directory
def extract_features(filename):
	# load the model
	model = VGG19()
	# re-structure the model
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	# load the photo
	image = load_img(filename, target_size=(224, 224))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the VGG model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature

### load tokenizer, model & image

In [ ]:
# load the tokenizer
tokenizer = load(open(tokenizer_path, 'rb'))
# pre-define the max sequence length (from training)
max_length = 34
# load the model
model = load_model('models/flickr8k/vgg19/saved-model-05.h5')
# load and prepare the photograph
photo = extract_features(image_path)

### predict & show caption

In [ ]:
# generate description
description = generate_desc(model, tokenizer, photo, max_length)
print(description)

## InceptionV3
### utilities

In [18]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

def extract_features(filename):
	# load the model
	model = InceptionV3()
	# re-structure the model
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	# load the photo
	image = load_img(filename, target_size=(299, 299))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the VGG model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature

### load tokenizer, model & image

In [ ]:
# load the tokenizer
tokenizer = load(open(tokenizer_path, 'rb'))
# pre-define the max sequence length (from training)
max_length = 34
# load the model
model = load_model('models/flickr8k/inceptionv3/saved-model-05.h5')
# load and prepare the photograph
photo = extract_features(image_path)

### predict & show caption

In [ ]:
# generate description
description = generate_desc(model, tokenizer, photo, max_length)
print(description)